In [55]:
import numpy as np
from scipy.ndimage import imread
from PIL import Image
import random
import cv2
import matplotlib.pyplot as plt

In [84]:
# Calculate transformation matrix
def make_dlt(x,x_,norm=False):
    
    
    (U, norm_x) = get_simMatrix(x)
    (T, norm_x_) = get_simMatrix(x_)
        
    norm_H = dlt(norm_x, norm_x_)
    
    if norm:
        return norm_H
    else:
        H = np.dot(np.dot(np.linalg.inv(T), norm_H), U)
        return H

def dlt(x1, x2):
    # Dimension check
    assert x1.shape == x2.shape
        
    # Covnstruct matrix A in equation A.H = 0
    zero = np.zeros(3)
    A1= zero
    A1= np.append(A1, - x2[0][2]*x1[0])
    A1= np.append(A1,   x2[0][1]*x1[0])
    A1= A1.reshape(1, len(A1))
    A2 = x2[0][2]*x1[0]
    A2 = np.append(A2,   zero)
    A2 = np.append(A2, - x2[0][0]*x1[0])
    A2 = A2.reshape(1, len(A2))    
    
    A = A1
    A = np.append(A, A2, axis=0)
    
    for i in range(1,len(x1)):
        
        A1= zero
        A1= np.append(A1, - x2[i][2]*x1[i])
        A1= np.append(A1,   x2[i][1]*x1[i])
        A1= A1.reshape(1, len(A1))
           
        
        A2 = x2[i][2]*x1[i]
        A2 = np.append(A2,   zero)
        A2 = np.append(A2, - x2[i][0]*x1[i])
        A2 = A2.reshape(1, len(A2))
        
        A = np.append(A, A1, axis=0)
        A = np.append(A, A2, axis=0)
    
    A = np.array(A)
    U, S, V = np.linalg.svd(A, full_matrices=False)
    H = V[np.argmin(S)].reshape(3, 3)
    H= H/H[2][2]
           
    return H

def get_simMatrix(X):
    centroid = np.mean(X, axis=0)
    x_normalized = X - centroid
    s = np.sqrt(2)/np.mean(np.sqrt(np.sum(x_normalized**2, axis=1)))
    x_normalized = x_normalized * s
    x_normalized = make_homog(x_normalized)
    sim_matrix= np.array([[s, 0, -s * centroid[0]],[0, s, -s * centroid[1]],[0, 0, 1]])

    return (sim_matrix, x_normalized)

def make_homog(x):
    return np.append(x, np.ones((len(x), 1)), axis=1)


def transform_(H,x_):
    x1_from_x = np.dot(H, np.transpose(x_[0])).reshape(1, 3)
    #maybe_result = []
    
    for x_row in x_[1:]:
        x1_from_x = np.append(x1_from_x, np.dot(H, np.transpose(x_row)).reshape(1, 3), axis=0)
    
    return x1_from_x

In [94]:
#Initializing
def RANSAC(im1,im2,n=15,k=100,t=1,T=5,bestfit = None,besterr = 1000000):
    
    for iteration in range(k):
        print iteration
        
        alsoinliers = []
        notmaybeinliers=None
        #Generate Random Set
        maybeinliers=random.sample(range(len(im1)),n)
        maybeinliers.sort()
        
        #NOT Inliers array
        notmaybeinliers=[]
        
        for i in range(len(im1)):
            if i not in maybeinliers:
                notmaybeinliers.append(i)
    
        notmaybeinliers.sort()
        notmaybeinliers = np.array(notmaybeinliers)
        
        #needed points
        impoints_1 = im1[maybeinliers]
        impoints_2 = im2[maybeinliers]

        #maybe model        
        maybe_model = make_dlt(impoints_1,impoints_2)
        maybe_model_ = make_dlt(impoints_2,impoints_1)
        
        x_homog = make_homog(impoints_1)
        x__homog = make_homog(impoints_2)
        
        maybe_result = transform_(maybe_model,x__homog)
        maybe_result_ = transform_(maybe_model_,x_homog)
        
        a,b = maybe_result.shape
        
        err = []
        
        for i in range(a):
            x = pow(maybe_result[i][0]-x_homog[i][0],2)
            y = pow(maybe_result[i][1]-x_homog[i][1],2)
            w = pow(maybe_result[i][0]-x_homog[i][0],2)
            
            x_ = pow(maybe_result_[i][0]-x__homog[i][0],2)
            y_ = pow(maybe_result_[i][1]-x__homog[i][1],2)
            w_ = pow(maybe_result_[i][0]-x__homog[i][0],2)
            
            partial_err = np.sqrt(x+y+w)
            partial_err_ = np.sqrt(x_+y_+w_)
            
            partial_err = np.sqrt(partial_err+partial_err_)
            
            err.append(partial_err)
        
        thiserr = 0
        err = np.array(err)  
        
        for j in range(len(err)):
            if err[j] < t:
                alsoinliers.append(j)
                thiserr = thiserr + err[j]
        
        
        #maybe a good mode
        if thiserr<besterr:
            bestfit = maybe_model
            besterror = thiserr
        
        if len(alsoinliers)>T:
            print "Consensus Found"
            return bestfit,maybeinliers
    
    assert bestfit is not None
        
    return bestfit,maybeinliers

In [79]:
def SIFT(img):
    sift = cv2.SIFT()
    keypoints = sift.detect(img,None)
    keypoints, descriptors = sift.compute(img,keypoints)
    out = cv2.drawKeypoints(img,keypoints)
    return out,keypoints

def keypoint2Array(kp):
    kp1_array_x = []
    kp1_array_y = []
    
    for point in kp1:
        kp1_array_x.append(point.pt[0])
        kp1_array_y.append(point.pt[1])
    
    return np.vstack((kp1_array_x, kp1_array_y)).T


In [80]:
# 0 means GrayScale
img1 = cv2.imread('PGPShelf/20150616_074411.jpg',0)
img2 = cv2.imread('PGPShelf/20150616_074444.jpg',0)
#cv2.imshow('Original GrayScale', img1)
#cv2.waitKey()
#plt.imshow(img1, cmap = 'gray', interpolation = 'bicubic')
sift1,kp1 = SIFT(img1)
kp_array_1 = keypoint2Array(kp1)

sift2,kp2 = SIFT(img2)
kp_array_2 = keypoint2Array(kp2)

In [107]:
print kp_array_1.shape
print kp_array_2.shape


H_sift,kp_sift = RANSAC(kp_array_1,kp_array_2,n = 1500,k = 20, T =  8000, t = 0.5)

kp_sift = np.array((kp_sift))
print kp_sift.shape

(26158, 2)
(26158, 2)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
(1500,)


In [111]:
new_kp_1 = kp_array_1[kp_sift]
print new_kp_1.shape
new_kp_2 = kp_array_2[kp_sift]

plt.imshow(img1, cmap = 'gray', interpolation = 'bicubic')
plt.scatter(new_kp_1[:,0],new_kp_1[:,1])
plt.show()

plt.imshow(img2, cmap = 'gray', interpolation = 'bicubic')
plt.scatter(new_kp_2[:,0],new_kp_2[:,1])
plt.show()

(1500, 2)


In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)